In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
import nltk
import re 
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from tqdm.auto import tqdm
tqdm.pandas()
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /home/yagor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_data = pd.read_csv('/home/yagor/Рабочий стол/BIV/bertembeded_logreg.csv', index_col=0)

In [3]:
train_data

,date,sum,text,category,proba
number,,,,,
1,07.11.2024,40500.00,За тур.поездку по договору №001 от 27.01.2023г,7,0.999575
2,07.11.2024,"32600,00",За оказание услуг по договору №53Б-02746 от 23...,7,0.999950
3,07.11.2024,4710-00,Оплата штрафа,8,0.923104
4,07.11.2024,30900-00,Лечение по договору №Д-00359/24 от 08.03.2025,7,0.999642
5,07.11.2024,13200.00,Оплата основного долга за период с 16.12.2024г...,6,0.995218
...,...,...,...,...,...
24996,07.11.2024,"3300000,00",Оплата по договору №Е01905 от 25.02.2023. сырь...,4,0.997016
24997,07.11.2024,"2270,00",гос.услуга,8,0.996888
24998,07.11.2024,1750000.00,Предоставление кредита по договору №Д-00803/03...,6,0.988926


In [4]:
train_data = train_data[train_data['proba']>=0.94]
train_data

,date,sum,text,category,proba
number,,,,,
1,07.11.2024,40500.00,За тур.поездку по договору №001 от 27.01.2023г,7,0.999575
2,07.11.2024,"32600,00",За оказание услуг по договору №53Б-02746 от 23...,7,0.999950
4,07.11.2024,30900-00,Лечение по договору №Д-00359/24 от 08.03.2025,7,0.999642
5,07.11.2024,13200.00,Оплата основного долга за период с 16.12.2024г...,6,0.995218
6,07.11.2024,4210.00,Оплата за Бульон Роллтон Домашний куриный 90г ...,5,0.999649
...,...,...,...,...,...
24995,07.11.2024,4560000-00,Оплата по договору процентного займа №46-04624...,0,0.998126
24996,07.11.2024,"3300000,00",Оплата по договору №Е01905 от 25.02.2023. сырь...,4,0.997016
24997,07.11.2024,"2270,00",гос.услуга,8,0.996888


In [5]:
train_data['summa'] = train_data['sum'].str.replace('[^\w]', '.', regex=True).astype(float)

/tmp/ipykernel_5805/825965816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['summa'] = train_data['sum'].str.replace('[^\w]', '.', regex=True).astype(float)


In [6]:
# train_data['label'] = train_data['category'].apply(lambda x: rename_dict[x])

In [7]:
test_data = pd.read_csv('/home/yagor/Рабочий стол/BIV/Кейс 1_for-teams/for-teams/payments_training.tsv', index_col=0, sep='\t', names = ['number',  'date',  'sum', 'text', 'category'])

In [8]:
test_data['text'].values[0]

'За участие в конференции в г. Майкоп по договору 88.367840-ДВ от 11 апреля 2026'

In [9]:
rename_dict = {'LOAN': 0,
 'REALE_STATE': 1,
 'LEASING': 2,
 'NOT_CLASSIFIED': 3,
 'NON_FOOD_GOODS': 4,
 'FOOD_GOODS': 5,
 'BANK_SERVICE': 6,
 'SERVICE': 7,
 'TAX': 8}

In [10]:
test_data['label'] = test_data['category'].apply(lambda x: rename_dict[x])

In [11]:
test_data

,date,sum,text,category,label
number,,,,,
1,07.11.2024,15300.00,За участие в конференции в г. Майкоп по догово...,SERVICE,7
2,07.11.2024,"40200,00",За оказание услуг по договору №79-02726В от 01...,SERVICE,7
3,07.11.2024,1440-00,Оплата за Порошок стиральный Ariel Color autom...,NON_FOOD_GOODS,4
4,07.11.2024,240000000-00,Возврат денежных средств по договору займа №04...,LOAN,0
5,07.11.2024,1360000.00,"Оплата Дог №452 от 13/03/2021, согл. Сч 0745-2...",NOT_CLASSIFIED,3
...,...,...,...,...,...
496,07.11.2024,2610.00,Оплата налогов,TAX,8
497,07.11.2024,31200-00,Комиссия за выполнение функций агента валютног...,BANK_SERVICE,6
498,07.11.2024,18200-00,За тур.поездку по договору №75-04243Г от 24/04...,SERVICE,7


In [12]:
# Получение русских стоп-слов
russian_stopwords = set(stopwords.words('russian'))
# Добавим специфичные для платежей стоп-слова
payment_stopwords = {'оплата', 'по', 'от', 'за', 'сумма', 'без', 'ндс', 'договор', 'счет', 'счёт'} #'оплата', 'по', 'от', 'за', 'сумма', 'без', 'ндс', 'договор', 'счет', 'счёт'
russian_stopwords.update(payment_stopwords)



In [13]:
def normalize(word):
    return morph.parse(word)[0].normal_form

In [14]:
def clean_text(text):
    """Базовая очистка текста"""
    if isinstance(text, str):
        # Приведение к нижнему регистру
        text = text.lower()
        # Удаление цифр и специальных символов, оставляя пробелы между словами
        text = re.sub(r'[^а-яёa-z\s]', ' ', text)
        text = " ".join([normalize(word) for word in text.split() if word not in russian_stopwords and len(word) > 2])
        # Удаление множественных пробелов
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    return ""

In [15]:
# Очистка текста
test_data['cleaned_text'] = test_data['text'].apply(clean_text)
# Очистка текста
train_data['cleaned_text'] = train_data['text'].apply(clean_text)

/tmp/ipykernel_5805/1183253440.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['cleaned_text'] = train_data['text'].apply(clean_text)


In [16]:
import torch
import transformers as ppb

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [17]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.AutoModel, ppb.AutoTokenizer, 'cointegrated/LaBSE-en-ru')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [18]:
# tokenized = train_data['cleaned_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [19]:
# import torch
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
# model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
# sentences = ["Hello World", "Привет Мир"]
# encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')
# with torch.no_grad():
#     model_output = model(**encoded_input)
# embeddings = model_output.pooler_output
# embeddings = torch.nn.functional.normalize(embeddings)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

### Step #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [20]:
# interested_labels = [rename_dict[i] for i in ['NON_FOOD_GOODS']] #, 'FOOD_GOODS'
# interested_labels

In [21]:
# new_mapping = {}
# ind = 0
# for i in df_grouped.index:
#     if rename_dict[i] in interested_labels:
#         new_mapping[rename_dict[i]] = ind
#         ind+=1
#     else:
#         new_mapping[rename_dict[i]] = len(interested_labels)

In [22]:
# new_mapping

In [23]:
# train_data['label_new'] = train_data['label'].apply(lambda x: new_mapping[x])

In [24]:
# train_data45 = train_data[train_data['label_new'].isin([0, 1])].copy()

In [25]:
# texts = train_data45['cleaned_text'].values
# target = train_data45['label_new'].values

In [26]:
test_data

,date,sum,text,category,label,cleaned_text
number,,,,,,
1,07.11.2024,15300.00,За участие в конференции в г. Майкоп по догово...,SERVICE,7,участие конференция майкоп договор апрель
2,07.11.2024,"40200,00",За оказание услуг по договору №79-02726В от 01...,SERVICE,7,оказание услуга договор
3,07.11.2024,1440-00,Оплата за Порошок стиральный Ariel Color autom...,NON_FOOD_GOODS,4,порошок стиральный ariel color automat счёт ав...
4,07.11.2024,240000000-00,Возврат денежных средств по договору займа №04...,LOAN,0,возврат денежный средство договор заём
5,07.11.2024,1360000.00,"Оплата Дог №452 от 13/03/2021, согл. Сч 0745-2...",NOT_CLASSIFIED,3,дог соглый сброс загрязнять вещество
...,...,...,...,...,...,...
496,07.11.2024,2610.00,Оплата налогов,TAX,8,налог
497,07.11.2024,31200-00,Комиссия за выполнение функций агента валютног...,BANK_SERVICE,6,комиссия выполнение функция агент валютный кон...
498,07.11.2024,18200-00,За тур.поездку по договору №75-04243Г от 24/04...,SERVICE,7,тур поездка договор


In [27]:
texts_train, target = train_data['cleaned_text'].values, train_data['category'].values
texts_test, target_test = test_data['cleaned_text'].values, test_data['label'].values

In [28]:
# texts_train, texts_test, y_train, y_test = train_test_split(texts,target,test_size=0.3,random_state=52, stratify=target)

In [29]:
texts_train_tokenized = [tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True) for x in texts_train]

In [30]:
texts_test_tokenized = [tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True) for x in texts_test]

In [31]:
def pad_texts(texts_tokenized):
    max_len = 0
    for i in texts_tokenized:
        if len(i) > max_len:
            max_len = len(i)

    padded = np.array([i + [0]*(max_len-len(i)) for i in texts_tokenized])
    return padded

In [32]:
padded_train = pad_texts(texts_train_tokenized)
padded_test = pad_texts(texts_test_tokenized)

In [33]:
def attention_mask(padded_text):
    return np.where(padded_text != 0, 1, 0)

In [34]:
attention_mask_train = attention_mask(padded_train)
attention_mask_train.shape

(19252, 31)

In [35]:
attention_mask_test = attention_mask(padded_test)
attention_mask_test.shape

(500, 29)

Now move the model to GPU and check if it is in `eval` mode.

In [36]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [37]:
model = model_class.from_pretrained(pretrained_weights)

Finally, process all the data with the BERT model:

In [38]:
model.eval()
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(55083, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

Cast the result to the numpy (e.g.) array:

In [39]:
import tqdm
torch.cuda.empty_cache()

In [40]:
output = []
batch_size = 16
for inx in tqdm.notebook.trange(0, attention_mask_train.shape[0], batch_size):
    batch = torch.tensor(padded_train[inx:inx+batch_size]).to(device)
    local_attention_mask = torch.tensor(attention_mask_train[inx:inx+batch_size]).to(device)

    with torch.no_grad():
        last_hidden_states = model(batch, attention_mask=local_attention_mask)[0][:, 0, :].cpu().numpy()
        output.append(last_hidden_states)

  0%|          | 0/1204 [00:00<?, ?it/s]

In [41]:
output_test = []
batch_size = 16
for inx in tqdm.notebook.trange(0, attention_mask_test.shape[0], batch_size):
    batch = torch.tensor(padded_test[inx:inx+batch_size]).to(device)
    local_attention_mask = torch.tensor(attention_mask_test[inx:inx+batch_size]).to(device)

    with torch.no_grad():
        last_hidden_states = model(batch, attention_mask=local_attention_mask)[0][:, 0, :].cpu().numpy()
        output_test.append(last_hidden_states)

  0%|          | 0/32 [00:00<?, ?it/s]

In [42]:
train_features = np.vstack(output)

In [43]:
test_features = np.vstack(output_test)

In [44]:
lr_clf = LogisticRegression(solver='lbfgs', max_iter=1000, C=2.5642)
lr_clf.fit(train_features, target)

LogisticRegression(C=2.5642, max_iter=1000)

In [58]:
import pickle

In [72]:
filename = '/home/yagor/Рабочий стол/BIV/biv_hack/models/linear_model1.sav'
# pickle.dump(lr_clf, open(filename, 'wb')) 
  
# load the model 
load_model = pickle.load(open(filename, 'rb')) 

In [64]:
load_model

LogisticRegression(C=2.5642, max_iter=1000)

In [ ]:
lr_clf = LogisticRegression(solver='lbfgs', max_iter=1000, C=2.5642)
lr_clf.fit(train_features, target)

In [48]:
train_features.shape

(19252, 768)

In [62]:
test_features.shape

(500, 768)

In [ ]:
y_pred = load_model.predict(test_features)

In [69]:
y_pred

array([7, 7, 4, 0, 3, 2, 4, 4, 5, 0, 0, 6, 7, 7, 0, 7, 5, 4, 8, 4, 2, 8,
       5, 7, 5, 2, 2, 5, 4, 5, 4, 7, 5, 7, 4, 6, 5, 5, 6, 2, 4, 8, 7, 5,
       4, 4, 6, 4, 2, 4, 7, 5, 5, 4, 5, 4, 5, 8, 4, 4, 5, 3, 7, 0, 5, 4,
       5, 8, 0, 6, 5, 7, 5, 4, 7, 5, 3, 8, 7, 1, 4, 6, 2, 5, 7, 7, 1, 3,
       8, 5, 4, 2, 7, 4, 7, 4, 0, 7, 4, 5, 1, 2, 7, 5, 4, 4, 7, 8, 2, 8,
       7, 7, 5, 4, 6, 6, 4, 4, 2, 8, 3, 7, 3, 5, 0, 6, 5, 5, 8, 8, 6, 5,
       0, 4, 3, 5, 7, 6, 6, 6, 0, 5, 6, 5, 0, 0, 5, 4, 4, 7, 5, 7, 6, 2,
       1, 3, 5, 5, 6, 4, 4, 4, 7, 2, 5, 7, 2, 0, 7, 8, 5, 4, 7, 3, 7, 4,
       4, 2, 0, 7, 8, 8, 2, 8, 6, 4, 7, 6, 2, 4, 0, 0, 6, 7, 4, 0, 1, 6,
       4, 2, 1, 7, 7, 4, 2, 5, 5, 5, 4, 7, 6, 4, 2, 5, 5, 2, 7, 1, 7, 1,
       5, 5, 4, 6, 0, 3, 6, 4, 5, 2, 2, 1, 5, 4, 4, 0, 6, 5, 7, 4, 7, 4,
       0, 7, 4, 0, 1, 8, 8, 6, 0, 3, 7, 6, 3, 8, 5, 7, 5, 4, 7, 7, 8, 4,
       4, 2, 0, 1, 8, 4, 4, 4, 7, 4, 4, 5, 5, 4, 7, 3, 4, 1, 8, 3, 4, 1,
       3, 5, 7, 1, 4, 4, 6, 4, 3, 4, 4, 3, 0, 6, 7,

In [67]:
# Step 5: Evaluate the Model
print("Classification Report:")
print(classification_report(target_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       1.00      1.00      1.00        27
           2       1.00      1.00      1.00        38
           3       1.00      1.00      1.00        23
           4       1.00      1.00      1.00        96
           5       1.00      1.00      1.00        90
           6       1.00      1.00      1.00        49
           7       1.00      1.00      1.00        88
           8       1.00      1.00      1.00        48

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



In [64]:
# Step 4: Make Predictions on the Test Set
y_pred = lr_clf.predict(test_features)

# Step 5: Evaluate the Model
print("Classification Report:")
print(classification_report(target_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       1.00      1.00      1.00        27
           2       1.00      1.00      1.00        38
           3       1.00      1.00      1.00        23
           4       1.00      1.00      1.00        96
           5       1.00      1.00      1.00        90
           6       1.00      1.00      1.00        49
           7       1.00      1.00      1.00        88
           8       1.00      1.00      1.00        48

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



In [77]:
id2label = {'LOAN': 0,
 'REALE_STATE': 1,
 'LEASING': 2,
 'NOT_CLASSIFIED': 3,
 'NON_FOOD_GOODS': 4,
 'FOOD_GOODS': 5,
 'BANK_SERVICE': 6,
 'SERVICE': 7,
 'TAX': 8}
label2id = {v:k for k, v in id2label.items()}

In [78]:
label2id

{0: 'LOAN',
 1: 'REALE_STATE',
 2: 'LEASING',
 3: 'NOT_CLASSIFIED',
 4: 'NON_FOOD_GOODS',
 5: 'FOOD_GOODS',
 6: 'BANK_SERVICE',
 7: 'SERVICE',
 8: 'TAX'}

In [79]:
df = pd.DataFrame({'labels': y_pred}, index=test_data.index.values)

In [82]:
df

,labels
1,7
2,7
3,4
4,0
5,3
...,...
496,8
497,6
498,7
499,5


In [83]:
df['labels'] = df['labels'].apply(lambda x: label2id[x])

In [84]:
df

,labels
1,SERVICE
2,SERVICE
3,NON_FOOD_GOODS
4,LOAN
5,NOT_CLASSIFIED
...,...
496,TAX
497,BANK_SERVICE
498,SERVICE
499,FOOD_GOODS


In [69]:
test_data.index

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            491, 492, 493, 494, 495, 496, 497, 498, 499, 500],
           dtype='int64', name='number', length=500)

In [85]:
df.to_csv('result.tsv', header=False, sep='\t')

In [88]:
column_with_text, columns_with_category = 'text', 'category'
try:
    test_data = pd.read_csv('/home/yagor/Рабочий стол/BIV/biv_hack/data/result.tsv', index_col=0, sep='\t', names = ['number',  columns_with_category])
    print(f"Файл {'result.tsv'} загружен")
except Exception as ex:
    print(ex)
    print('Загрузите Ваш .tsv файл в правильном формате')

Файл result.tsv загружен


In [89]:
test_data

,category
number,
1,SERVICE
2,SERVICE
3,NON_FOOD_GOODS
4,LOAN
5,NOT_CLASSIFIED
...,...
496,TAX
497,BANK_SERVICE
498,SERVICE


In [87]:
test_data

,category
number,
1,SERVICE
2,SERVICE
3,NON_FOOD_GOODS
4,LOAN
5,NOT_CLASSIFIED
...,...
496,TAX
497,BANK_SERVICE
498,SERVICE
